In [1]:
import pandas as pd
import numpy as np
import talib
from binance.client import Client
from datetime import datetime


In [2]:
api_key = 'NSLpNhxFVqfh3zju'
api_secret = 'h6dXUoCM8doHBPV6g4g'

client = Client(api_key, api_secret)


In [4]:
def get_data(symbol, interval, start_time, end_time):
    klines = client.get_historical_klines(symbol, interval, start_time, end_time)
    data = pd.DataFrame(klines, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

    data['open_time'] = pd.to_datetime(data['open_time'], unit='ms')
    data['close_time'] = pd.to_datetime(data['close_time'], unit='ms')

    for col in ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']:
        data[col] = data[col].astype(float)

    data.set_index('open_time', inplace=True, drop=False)

    return data


In [5]:
def add_indicators(df):
    df['sma_15'] = talib.SMA(df['close'], timeperiod=15)
    df['sma_33'] = talib.SMA(df['close'], timeperiod=33)
    df['sma_56'] = talib.SMA(df['close'], timeperiod=56)
    df['sma_112'] = talib.SMA(df['close'], timeperiod=112)
    df['sma_224'] = talib.SMA(df['close'], timeperiod=224)
    df['sma_448'] = talib.SMA(df['close'], timeperiod=448)
    df['adx'] = talib.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    df['macd'], df['macd_signal'], df['macd_hist'] = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)

    return df


In [6]:
symbol = 'BTCUSDT'
interval = Client.KLINE_INTERVAL_1MINUTE
start_time = '1 year ago UTC'
end_time = 'now UTC'

data = get_data(symbol, interval, start_time, end_time)
data_with_indicators = add_indicators(data)


In [7]:
data_with_indicators.fillna(method='bfill', inplace=True)


In [8]:
def split_data(data, test_ratio=0.2):
    split_index = int(len(data) * (1 - test_ratio))
    train_data = data.iloc[:split_index]
    test_data = data.iloc[split_index:]

    return train_data, test_data

train_data, test_data = split_data(data_with_indicators)


In [9]:
def reshape_data(data, sequence_length):
    x = []
    y = []

    for i in range(len(data) - sequence_length):
        x.append(data.iloc[i:i + sequence_length].values)
        y.append(data.iloc[i + sequence_length]['close'])

    x = np.array(x)
    y = np.array(y)

    return x, y

sequence_length = 60
x_train, y_train = reshape_data(train_data, sequence_length)
x_test, y_test = reshape_data(test_data, sequence_length)


In [10]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

# LSTM 모델 생성
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')


In [17]:
train_data = train_data.drop(columns=['open_time', 'close_time'])
test_data = test_data.drop(columns=['open_time', 'close_time'])

In [ ]:
x_train, y_train = reshape_data(train_data, sequence_length)
x_test, y_test = reshape_data(test_data, sequence_length)

history = model.fit(x_train, y_train, epochs=100, batch_size=32)

In [ ]:
y_pred = model.predict(x_test)

# 성능 평가를 위한 추가적인 지표를 계산할 수 있습니다 (예: MSE, MAE 등).

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 5))
plt.plot(y_test, label='True')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.show()
